<a href="https://colab.research.google.com/github/gemepolimi/gemepolimi/blob/main/Custom_GHCN_Daily_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from io import StringIO

# ── UPLOAD ────────────────────────────────────────────────────────────────────
uploaded = files.upload()
filename = list(uploaded.keys())[0]
content  = uploaded[filename].decode("utf-8")

# ── PARSE CSV — adjust if your columns differ ─────────────────────────────────
df = pd.read_csv(StringIO(content))
print("Columns:", df.columns.tolist())
print(df.head(3))

Saving 4231562.csv to 4231562.csv
Saving 4231653.csv to 4231653.csv
Columns: ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'PRCP', 'PRCP_ATTRIBUTES']
       STATION           NAME  LATITUDE  LONGITUDE  ELEVATION        DATE  \
0  ET000063332  BAHAR DAR, ET      11.6     37.417     1770.0  1961-07-08   
1  ET000063332  BAHAR DAR, ET      11.6     37.417     1770.0  1961-07-09   
2  ET000063332  BAHAR DAR, ET      11.6     37.417     1770.0  1961-07-10   

   PRCP PRCP_ATTRIBUTES  
0   4.6             ,,Q  
1   0.3             ,,Q  
2   1.3             ,,Q  


In [ ]:
# ── AFTER confirming columns, set these: ─────────────────────────────────────
# Expected columns: STATION, NAME (or STATION_NAME), DATE, PRCP (or RF / RAINFALL)
# DATE format: YYYY-MM-DD  or  YYYYMMDD

STATION_COL  = "NAME"   # <-- change to your station name column
DATE_COL     = "DATE"           # <-- change to your date column
RAIN_COL     = "PRCP"           # <-- change to your rainfall value column

MONTHS = ["J","F","M","A","M","J","J","A","S","O","N","D"]
GREEN  = "C4D9A8"
BLUE   = "D9E8F5"
WHITE  = "FFFFFF"
BLACK  = "000000"
GREY   = "AAAAAA"

# ── STYLES ────────────────────────────────────────────────────────────────────
def mk_fill(c): return PatternFill("solid", fgColor=c)
def mk_font(bold=False, size=10): return Font(color=BLACK, bold=bold, size=size, name="Calibri")
def mk_border():
    s = Side(style="thin", color=GREY)
    return Border(left=s, right=s, top=s, bottom=s)
def mk_align(): return Alignment(horizontal="center", vertical="center", wrap_text=True)

def style(cell, bg=WHITE, bold=False, size=10, fmt=None):
    cell.fill      = mk_fill(bg)
    cell.font      = mk_font(bold, size)
    cell.border    = mk_border()
    cell.alignment = mk_align()
    if fmt: cell.number_format = fmt

def set_col_widths(ws, max_col):
    for c in range(1, max_col + 1):
        ltr = get_column_letter(c)
        if c == 1:          ws.column_dimensions[ltr].width = 15
        elif c == max_col:  ws.column_dimensions[ltr].width = 16
        else:               ws.column_dimensions[ltr].width = 7

def set_row_heights(ws, max_row, summary_row=None):
    ws.row_dimensions[1].height = 20
    for r in range(2, max_row + 1):
        ws.row_dimensions[r].height = 15
    if summary_row:
        ws.row_dimensions[summary_row].height = 18

# ── WRITE ANNUAL SUMMARY SHEET ────────────────────────────────────────────────
def write_annual_sheet(wb, station_df):
    ws = wb.create_sheet("Annual & Monthly Total", 0)
    ws.freeze_panes = "B2"

    # Header
    headers = ["Year"] + MONTHS + ["Total Annual"]
    for c, h in enumerate(headers, 1):
        cell = ws.cell(row=1, column=c, value=h)
        style(cell, bg=GREEN, bold=True, size=10)

    years = sorted(station_df["YEAR"].unique())
    for row_idx, year in enumerate(years, 2):
        year_df = station_df[station_df["YEAR"] == year]
        monthly = []
        for m in range(1, 13):
            val = year_df[year_df["MONTH"] == m][RAIN_COL].sum()
            monthly.append(round(val, 2))
        total = round(sum(monthly), 2)
        row_data = [year] + monthly + [total]

        for c, val in enumerate(row_data, 1):
            cell = ws.cell(row=row_idx, column=c, value=val)
            is_last = c == len(row_data)
            bg = BLUE if is_last else WHITE
            bold = is_last
            style(cell, bg=bg, bold=bold, fmt="0.00" if c > 1 else None)

    # Summary row (median)
    summary_row = len(years) + 2
    ws.cell(row=summary_row, column=1, value="Monthly Total")
    style(ws.cell(row=summary_row, column=1), bg=BLUE, bold=True)

    for c in range(2, 15):
        vals = [ws.cell(row=r, column=c).value for r in range(2, summary_row)
                if ws.cell(row=r, column=c).value is not None]
        median_val = round(sorted(vals)[len(vals)//2], 2) if vals else 0
        cell = ws.cell(row=summary_row, column=c, value=median_val)
        style(cell, bg=BLUE, bold=True, fmt="0.00")

    set_col_widths(ws, 14)
    set_row_heights(ws, summary_row, summary_row)
    ws.sheet_properties.tabColor = "C4D9A8"

# ── WRITE DAILY YEAR SHEET ────────────────────────────────────────────────────
def write_year_sheet(wb, year, year_df):
    ws = wb.create_sheet(str(year))
    ws.freeze_panes = "B2"

    headers = ["Day"] + MONTHS + ["Total Annual"]
    for c, h in enumerate(headers, 1):
        cell = ws.cell(row=1, column=c, value=h)
        style(cell, bg=GREEN, bold=True, size=10)

    for day in range(1, 32):
        row_idx = day + 1
        ws.cell(row=row_idx, column=1, value=day)
        style(ws.cell(row=row_idx, column=1), bg=WHITE, bold=False)

        daily_total = 0
        for m_idx, m in enumerate(range(1, 13), 2):
            day_data = year_df[(year_df["MONTH"] == m) & (year_df["DAY"] == day)]
            val = round(day_data[RAIN_COL].sum(), 2) if not day_data.empty else 0.0
            daily_total += val
            cell = ws.cell(row=row_idx, column=m_idx, value=val)
            style(cell, bg=WHITE, fmt="0.00")

        # Total Annual column (last)
        last_cell = ws.cell(row=row_idx, column=14, value=round(daily_total, 2))
        style(last_cell, bg=BLUE, bold=True, fmt="0.00")

    # Monthly Total row
    summary_row = 33
    ws.cell(row=summary_row, column=1, value="Monthly Total")
    style(ws.cell(row=summary_row, column=1), bg=BLUE, bold=True)

    grand_total = 0
    for m_idx, m in enumerate(range(1, 13), 2):
        total = round(year_df[year_df["MONTH"] == m][RAIN_COL].sum(), 2)
        grand_total += total
        cell = ws.cell(row=summary_row, column=m_idx, value=total)
        style(cell, bg=BLUE, bold=True, fmt="0.00")

    last = ws.cell(row=summary_row, column=14, value=round(grand_total, 2))
    style(last, bg=BLUE, bold=True, fmt="0.00")

    set_col_widths(ws, 14)
    set_row_heights(ws, summary_row, summary_row)
    ws.sheet_properties.tabColor = "C4D9A8"

# ── MAIN ──────────────────────────────────────────────────────────────────────
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df["YEAR"]   = df[DATE_COL].dt.year
df["MONTH"]  = df[DATE_COL].dt.month
df["DAY"]    = df[DATE_COL].dt.day
df[RAIN_COL] = pd.to_numeric(df[RAIN_COL], errors="coerce").fillna(0)

for station in df[STATION_COL].unique():
    station_df = df[df[STATION_COL] == station].copy()
    safe_name  = "".join(c if c.isalnum() or c in " _-" else "_" for c in str(station))
    out_file   = f"{safe_name}.xlsx"

    wb = openpyxl.Workbook()
    wb.remove(wb.active)  # remove default sheet

    write_annual_sheet(wb, station_df)
    for year in sorted(station_df["YEAR"].unique()):
        write_year_sheet(wb, year, station_df[station_df["YEAR"] == year])

    wb.save(out_file)
    files.download(out_file)
    print(f"Downloaded: {out_file}")